In [1]:
import numpy as np
import xarray as xr
import matplotlib.pylab as plt
import cmocean as cm
%matplotlib inline
import gsw
import seawater as sw

In [2]:
output_path="/home1/datahome/jmartine/models/nemo/tests/MY_ICE_CANAL/EXP00/init/"
# output_path="/home/jmartine/github/CANAL_nemo/EXP00/init/"

In [3]:
ngrid_x = 501
ngrid_y = 253
ngrid_z = 101

In [4]:
δx=2
δy=2
δz=1
max_depth=800

In [5]:
x=np.linspace(0,ngrid_x*δx,ngrid_x)
y=np.linspace(0,ngrid_y*δy,ngrid_y)

In [6]:
from utils import *

In [7]:
depth,_ = compute_depth_prof(ngrid_z,max_depth)

In [8]:
Ys,Zs = np.meshgrid(y,depth) 

In [9]:
# Width of gradient horizontally
w  = 100/2.5
Lf = w
# Width of gradient vertically
dh=200#75

In [10]:
#Depth location of gradient:
H1=75
#horizontal location of gradient:
L  = (ngrid_y*δy)/2

In [11]:
# Function to create gradient
horizontal_depend_tanh =  (smooth_tanh(-Zs,dh,-H1) +  (smooth_tanh(Ys,Lf,L)) * smooth_tanh(-Zs,dh,-H1)  + abs( smooth_tanh(-Zs,dh,-H1) -1 ) * 1.5) - 1.5

In [12]:
# Function to scale gradient
def scale_gradient(gradient, scaling):
    gradient   = gamma * (horizontal_depend_tanh) #* (-1 + depth_depend_tanh)
    return gradient 
# Scale gradient by a factor gamma
gamma =  1
test_grad = scale_gradient(horizontal_depend_tanh,gamma)

In [13]:
# Init conditions of ocean state:
salt_0 = 33.5 #Salinity

ft_s= gsw.t_freezing(salt_0,0,0)
temp_0 = np.max(ft_s) + 0.5 #Freezing point

In [14]:
init_sal= salt_0 + smooth_tanh(Zs,330,110) + 0.5*(Zs/max_depth)*smooth_tanh(Zs,330,110)
init_temp= temp_0 + smooth_tanh(Zs,330,110) - 0.5*(Zs/max_depth)*smooth_tanh(Zs,330,110)

In [15]:
def generate_noise(nx,ny,nz,scale,lz):
    noise = np.zeros((nz,ny,nx))
    noise[0:lz,:,:] = (scale * np.random.rand(1,lz,ny,nx)) - (scale/2)
    return noise

In [17]:
scalings = [0,0.5,1]

nemo_version=4

temp_dict={}
salt_dict={}

dataset_list = {}

for gamma in scalings:
    gradient = scale_gradient(horizontal_depend_tanh,gamma)

    salt_dict[gamma] = ( - 1 * gradient + init_sal)
#     temp_dict[gamma] = ( gradient * temp_0 + init_temp) # Gradient in temperature
    temp_dict[gamma] = (init_temp)
    
    print(nemo_version)
    if nemo_version >= 4:
        pres = sw.eos80.pres(depth,0)
        practical_salt = gsw.conversions.SP_from_SA(salt_dict[gamma],pres[:,None],0,80)
        model_temp = gsw.conversions.pt0_from_t(init_sal,temp_dict[gamma],pres[:,None])
        model_temp = gsw.conversions.CT_from_pt(init_sal,model_temp)
        ρ = gsw.density.rho(practical_salt, model_temp, pres[:,None])
        
    else:
        pres = sw.eos80.pres(depth,0)
        practical_salt = gsw.conversions.SP_from_SA(salt_dict[gamma],pres[:,None],0,80)
        model_temp = gsw.conversions.pt0_from_t(init_sal,temp_dict[gamma],pres[:,None])
        ρ = gsw.density.rho(practical_salt, model_temp, pres[:,None])


    noise = generate_noise(ngrid_x,ngrid_y,ngrid_z,0.05,40)
    
    
    ds_tem=np.ones((ngrid_z,ngrid_y,ngrid_x))
    ds_sal=np.ones((ngrid_z,ngrid_y,ngrid_x))
    ds_dens=np.ones((ngrid_z,ngrid_y,ngrid_x))
    
    ds_tem_init = ds_tem*model_temp[:,:,None]
    ds_sal_init = ds_sal*practical_salt[:,:,None] + noise
    ds_dens_init = ds_dens*ρ[:,:,None]
    
    dsout = xr.Dataset(coords=dict(x=(["x"], x),xu=(["xu"], x[:-1]+0.5), y=(["y"], y),yu=(["yu"], y[:-1]+0.5),z=(['z'],depth)))

    dsout['votemper'] = (('z','y','x'),ds_tem_init)
    dsout['vosaline'] = (('z','y','x'),ds_sal_init)
    dsout['vorho']    = (('z','y','x'),ds_dens_init)
    
    dsout = dsout.expand_dims(time=6)
    dsout.assign_coords({'z':depth,'x':x,'y':y})
    dataset_list[gamma]=dsout
    
    dsout.to_netcdf(output_path+'Channel_Oce_pt_noise_pS_1000_5000_grid_freezing_point_75m_lin_strat_depth_6months_magnitude_conserved_contents_no_TEMP_grad_{0}_nemo{1}.nc'.format(gamma,nemo_version),unlimited_dims={'time':True})
#     dsout.to_netcdf(output_path+'Channel_Oce_pt_noise_pS_1000_5000_grid_freezing_point_75m_lin_strat_depth_6months_magnitude_conserved_contents_{0}_nemo{1}.nc'.format(gamma,nemo_version),unlimited_dims={'time':True})

4
4
4


In [18]:
print((dataset_list[0].votemper.isel(time=0,z=0,y=50) - dataset_list[0].votemper.isel(time=0,z=0,y=200)).mean())
print((dataset_list[0].vosaline.isel(time=0,z=0,y=50) - dataset_list[0].vosaline.isel(time=0,z=0,y=200)).mean())
print((dataset_list[0].vorho.isel(time=0,z=0,y=50) - dataset_list[0].vorho.isel(time=0,z=0,y=200)).mean())

<xarray.DataArray 'votemper' ()> Size: 8B
array(0.)
Coordinates:
    z        float64 8B 0.0
<xarray.DataArray 'vosaline' ()> Size: 8B
array(0.00103321)
Coordinates:
    z        float64 8B 0.0
<xarray.DataArray 'vorho' ()> Size: 8B
array(0.)
Coordinates:
    z        float64 8B 0.0


In [19]:
print((dataset_list[0.5].votemper.isel(time=0,z=0,y=50)-dataset_list[0.5].votemper.isel(time=0,z=0,y=200)).mean())
print((dataset_list[0.5].vosaline.isel(time=0,z=0,y=50) - dataset_list[0.5].vosaline.isel(time=0,z=0,y=200)).mean())
print((dataset_list[0.5].vorho.isel(time=0,z=0,y=50) - dataset_list[0.5].vorho.isel(time=0,z=0,y=200)).mean())

<xarray.DataArray 'votemper' ()> Size: 8B
array(0.)
Coordinates:
    z        float64 8B 0.0
<xarray.DataArray 'vosaline' ()> Size: 8B
array(0.49406168)
Coordinates:
    z        float64 8B 0.0
<xarray.DataArray 'vorho' ()> Size: 8B
array(0.3978758)
Coordinates:
    z        float64 8B 0.0


In [20]:
print((dataset_list[1].votemper.isel(time=0,z=0,y=50)-dataset_list[1].votemper.isel(time=0,z=0,y=200)).mean())
print((dataset_list[1].vosaline.isel(time=0,z=0,y=50) - dataset_list[1].vosaline.isel(time=0,z=0,y=200)).mean())
print((dataset_list[1].vorho.isel(time=0,z=0,y=50) - dataset_list[1].vorho.isel(time=0,z=0,y=200)).mean())

<xarray.DataArray 'votemper' ()> Size: 8B
array(0.)
Coordinates:
    z        float64 8B 0.0
<xarray.DataArray 'vosaline' ()> Size: 8B
array(0.98663562)
Coordinates:
    z        float64 8B 0.0
<xarray.DataArray 'vorho' ()> Size: 8B
array(0.79575163)
Coordinates:
    z        float64 8B 0.0
